In [1]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image
import cv2

In [ ]:
MODEL_PATH = '../models/mobile_net_v2_pcb.h5'
TEST_IMAGE_PATH = '../data/processed/validation/defect/sample_defect.jpg'
LAST_CONV_LAYER = 'out_relu'

model = load_model(MODEL_PATH)

In [ ]:
def make_gradcam_heatmap(img_array, model, last_conv_layer_name):
    grad_model = tf.keras.models.Model(
        [model.inputs],
        [model.get_layer(last_conv_layer_name).output, model.output]
    )

    with tf.GradientTape() as tape:
        conv_outputs, predictions = grad_model(img_array)
        loss = predictions[:, 0]

    grads = tape.gradient(loss, conv_outputs)
    pooled_grads = tf.reduce_mean(grads, axis=(0, 1, 2))

    conv_outputs = conv_outputs[0]
    heatmap = conv_outputs @ pooled_grads[..., tf.newaxis]
    heatmap = tf.squeeze(heatmap)
    heatmap = tf.maximum(heatmap, 0) / tf.math.reduce_max(heatmap)

    return heatmap.numpy()

def overlay_heatmap(img_path, heatmap, alpha=0.4):
    img = cv2.imread(img_path)
    img = cv2.resize(img, (224, 224))

    heatmap = cv2.resize(heatmap, (img.shape[1], img.shape[0]))
    heatmap = np.uint8(255 * heatmap)
    heatmap = cv2.applyColorMap(heatmap, cv2.COLORMAP_JET)

    superimposed = cv2.addWeighted(img, 1-alpha, heatmap, alpha, 0)
    return img, heatmap, superimposed

In [ ]:
img = image.load_img(TEST_IMAGE_PATH, target_size=(224, 224))
img_array = image.img_to_array(img)
img_array = np.expand_dims(img_array, axis=0) / 255.0

In [ ]:
prediction = model.predict(img_array)[0][0]
pred_class = "Defect" if prediction > 0.5 else "Pass"
print(f"Prediction: {pred_class} (Score: {prediction:.4f})")

In [ ]:
# Grad-CAM
heatmap = make_gradcam_heatmap(img_array, model, LAST_CONV_LAYER)
original, heatmap_img, overlay = overlay_heatmap(TEST_IMAGE_PATH, heatmap)

In [ ]:
plt.figure(figsize=(15, 5))

plt.subplot(1, 3, 1)
plt.imshow(cv2.cvtColor(original, cv2.COLOR_BGR2RGB))
plt.title(f"Original\n{pred_class}")
plt.axis('off')

plt.subplot(1, 3, 2)
plt.imshow(heatmap_img)
plt.title("Grad-CAM Heatmap")
plt.axis('off')

plt.subplot(1, 3, 3)
plt.imshow(cv2.cvtColor(overlay, cv2.COLOR_BGR2RGB))
plt.title("Overlay")
plt.axis('off')

plt.tight_layout()
plt.show()

print("Grad-CAM analysis complete")